In [1]:
import json
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding


Using TensorFlow backend.


In [10]:
vqa_train = json.load(open('C:/VQA/data/vqa_train.json'))
questions =[]
answers =[]

for i in range(len(vqa_train)):
    questions.append(vqa_train[i]['question'][:-1])
    answers.append(vqa_train[i]['ans'])
questions=questions[:1000]
answers = answers[:1000]

In [11]:
vocab=[]
from nltk.tokenize import word_tokenize
for que in questions:
    tokenize_word = word_tokenize(que)
    for word in tokenize_word:
        vocab.append(word)
for ans in answers:
    tokenize_word = word_tokenize(ans)
    for word in tokenize_word:
        vocab.append(word)

        

In [12]:
vocab = set(vocab)
vocab_len = len(vocab)

In [13]:
embedded_questions = [one_hot(ques,vocab_len) for ques in questions]
print(questions[0])
print(questions[1])
print(embedded_questions[0])
print(embedded_questions[1])


embedded_ans = [one_hot(ans,vocab_len) for ans in answers]
print(embedded_ans[0])
print(embedded_ans[1])


What is this photo taken looking through
What position is this man playing
[1129, 593, 864, 203, 435, 761, 887]
[1129, 100, 593, 864, 227, 57]
[110]
[760]


In [14]:
question_count = lambda sentence: len(word_tokenize(sentence))
longest_question = max(questions, key=question_count)
length_long_question = len(word_tokenize(longest_question))
print('Longest sentence ='+str(length_long_question))
ans_count = lambda sentence: len(word_tokenize(sentence))
longest_ans = max(answers, key=ans_count)
length_long_ans = len(word_tokenize(longest_ans))
print('Longest sentence ='+str(length_long_ans))




Longest sentence =17
Longest sentence =9


In [49]:
padded_questions = pad_sequences(embedded_questions, length_long_question, padding='post')
print(padded_questions.shape)
print(padded_questions[0])
print(padded_questions[1])
padded_answers = pad_sequences(embedded_ans, vocab_len, padding='post')

#padded_answers =  [item[0] for item in padded_answers]
print(padded_answers.shape)

(1000, 17)
[1129  593  864  203  435  761  887    0    0    0    0    0    0    0
    0    0    0]
[1129  100  593  864  227   57    0    0    0    0    0    0    0    0
    0    0    0]
(1000, 1216)


In [52]:

model = Sequential()
model.add(Embedding(vocab_len,32,input_length=length_long_question))
model.add(LSTM(256,dropout=0.2))
#model.add(Flatten())
model.add(Dense(vocab_len,activation='sigmoid'))

In [53]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 17, 32)            38912     
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               295936    
_________________________________________________________________
dense_3 (Dense)              (None, 1216)              312512    
Total params: 647,360
Trainable params: 647,360
Non-trainable params: 0
_________________________________________________________________
None


In [57]:
model.fit(padded_questions,padded_answers,epochs=100,verbose=1)

Epoch 1/100
1000/1000 [==============================] - 3s 3ms/step - loss: -278.9249 - acc: 0.9967
Epoch 2/100
1000/1000 [==============================] - 3s 3ms/step - loss: -282.7874 - acc: 0.9967
Epoch 3/100
1000/1000 [==============================] - 3s 3ms/step - loss: -286.6269 - acc: 0.9967
Epoch 4/100
1000/1000 [==============================] - 3s 3ms/step - loss: -290.4801 - acc: 0.9967
Epoch 5/100
1000/1000 [==============================] - 3s 3ms/step - loss: -294.3026 - acc: 0.9967
Epoch 6/100
1000/1000 [==============================] - 3s 3ms/step - loss: -298.1439 - acc: 0.9967
Epoch 7/100
1000/1000 [==============================] - 3s 3ms/step - loss: -302.0284 - acc: 0.9967
Epoch 8/100
1000/1000 [==============================] - 3s 3ms/step - loss: -305.9048 - acc: 0.9967
Epoch 9/100
1000/1000 [==============================] - 3s 3ms/step - loss: -309.7786 - acc: 0.9967
Epoch 10/100
1000/1000 [==============================] - ETA: 0s - loss: -313.8665 - acc: 

1000/1000 [==============================] - 3s 3ms/step - loss: -583.8566 - acc: 0.9967
Epoch 81/100
1000/1000 [==============================] - 3s 3ms/step - loss: -587.7221 - acc: 0.9967
Epoch 82/100
1000/1000 [==============================] - 3s 3ms/step - loss: -591.5919 - acc: 0.9967
Epoch 83/100
1000/1000 [==============================] - 3s 3ms/step - loss: -595.4086 - acc: 0.9967: 1s
Epoch 84/100
1000/1000 [==============================] - 3s 3ms/step - loss: -599.2760 - acc: 0.9967
Epoch 85/100
1000/1000 [==============================] - 3s 3ms/step - loss: -603.1872 - acc: 0.9967
Epoch 86/100
1000/1000 [==============================] - 3s 3ms/step - loss: -607.0277 - acc: 0.9967
Epoch 87/100
1000/1000 [==============================] - 3s 3ms/step - loss: -610.9134 - acc: 0.9967
Epoch 88/100
1000/1000 [==============================] - 3s 3ms/step - loss: -614.7928 - acc: 0.9967
Epoch 89/100
1000/1000 [==============================] - 3s 3ms/step - loss: -618.6714 - a

In [55]:
loss, accuracy = model.evaluate(padded_questions, padded_answers, verbose=1)
print('Accuracy: %f' % (accuracy*100))

1000/1000 [==============================] - 1s 897us/step
Accuracy: 99.670953


In [56]:
import numpy as np
test_question =padded_questions[0].reshape((1,-1))
print(test_question)
# test_question  =np.reshape(test_question,(1,))
print(test_question.shape)
test_output = model.predict(test_question,verbose=1)
print(test_output)
test_output = np.argmax(test_output.reshape((-1,1)))
print(test_output)

[[1129  593  864  203  435  761  887    0    0    0    0    0    0    0
     0    0    0]]
(1, 17)
1/1 [==============================] - 0s 215ms/step
[[1.00000000e+00 1.00000000e+00 1.00000000e+00 ... 1.29342079e-05
  7.74860382e-06 1.04010105e-05]]
0
